In [1]:
%matplotlib notebook
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tif
import json

In [2]:
def read_tif_files():
    '''function to read out tif files in directory'''
    #usr_in = input('Please give path to where is your data located at:  ')
    f_list = list(glob.iglob('/Users/timothyliu/Desktop/Globus/multimodal/data3_Textured/**/*.tif', recursive=True))
    dark_list = list(glob.iglob('/Users/timothyliu/Desktop/Globus/multimodal/data3_Textured/**/*dark*.tif', recursive=True))
    for el in f_list:
        if el in dark_list:
            f_list.remove(el)
    #out_list = [f for f in f_list if f not in dark_list]
    print('number of total files = {}'.format(len(list(f_list))))
    print('number of dark files = {}'.format(len(list(dark_list))))
    return list(f_list)
# doesn't have to do this step, once we have f_list
f_list = read_tif_files()
#print(f_list)
with open('/Users/timothyliu/hackathon/texture_f_list.txt', 'w') as f:
    json.dump(f_list, f)

number of total files = 172
number of dark files = 20


In [3]:
# collect clean data
std_list = list(glob.glob('/Users/timothyliu/Desktop/Globus/multimodal/clean_data/*.tif'))
dark_list = []
for el in std_list:
    if 'dark' in el:
        dark_list.append(el)
        std_list.remove(el)
std_list

['/Users/timothyliu/Desktop/Globus/multimodal/clean_data/CeO2_calib-00000.tif',
 '/Users/timothyliu/Desktop/Globus/multimodal/clean_data/Ni_300K-00044.tif',
 '/Users/timothyliu/Desktop/Globus/multimodal/clean_data/Ni_STD_300K-00538.tif',
 '/Users/timothyliu/Desktop/Globus/multimodal/clean_data/nickel-00000.raw.tif',
 '/Users/timothyliu/Desktop/Globus/multimodal/clean_data/nickel_296K-00002.tif',
 '/Users/timothyliu/Desktop/Globus/multimodal/clean_data/nickelstandard_100K-00028.tif']

In [4]:
add_list = list(glob.glob('/Users/timothyliu/Desktop/Globus/multimodal/clean_data/**/*.tif'))
with open('/Users/timothyliu/hackathon/std_f_list.txt', 'w') as std_f:
    json.dump(std_list, std_f)
with open('/Users/timothyliu/hackathon/extra_std_f_list.txt', 'w') as add_f:
    json.dump(add_list, add_f)

In [6]:
# start to work
os.chdir('/Users/timothyliu/hackathon/')
# read in json file
with open('texture_f_list.txt','r') as f:
    f_list = json.load(f)
with open('std_f_list.txt','r') as f:
    std_list = json.load(f)

In [7]:
# brutally corp images
def quarter_img(f_name, center=1024, width=200):
    img = tif.imread(f_name)
    img_list = []
    for i in np.linspace(-1,1,2):
        for j in np.linspace(-1,1,2):
            lower_x = min(center+int(i)*width, center)
            upper_x = max(center+int(i)*width, center)
            lower_y = min(center+int(j)*width, center)
            upper_y = max(center+int(j)*width, center)
            img_list.append(img[lower_x:upper_x, lower_y:upper_y])
    return img_list

def crop_img(f_name, center=1024, width=200):
    img = tif.imread(f_name)
    img_list = []
    img_list.append(img[center-width:center+width, center-width:center+width])
    return img_list

In [24]:
# loop over images and create np array of texture data
def file_dict(f_list):
    out_dict = {}
    for i in range(len(f_list)):
        out_dict.update({str(i):os.path.basename(f_list[i])})
        #print(os.path.basename(f_list[i]))
    return out_dict
        
def build_quarter_img_list(f_list):
    import tifffile as tif
    out_list = []
    #f_out_dict = []
    _img = tif.imread(f_list[1])
    num_obs = len(f_list)
    (x_dim, y_dim) = np.shape(_img)
    for i in range(len(f_list)):
        quarter_imgs = quarter_img(f_list[i])
        for el in quarter_imgs:
            out_list.append(el)
            #f_out_dict.append(str(i))
    out = np.array(out_list)
    return out #,f_out_list

def build_full_img_list(f_list):
    import tifffile as tif
    out_list = []
    _img = tif.imread(f_list[1])
    num_obs = len(f_list)
    (x_dim, y_dim) = np.shape(_img)
    for f in f_list:
        img = crop_img(f)
        out_list.append(img)
    _out = np.array(out_list)
    out = np.squeeze(_out)
    return out #,f_list

In [25]:
# quarter_imgs
quarter_texture_data_list = build_quarter_img_list(f_list)
quarter_standard_list = build_quarter_img_list(std_list)
quarter_extra_standard_list = build_quarter_img_list(add_list)

# save data
#os.makedirs('/Users/timothyliu/hackathon/quarter/', exist_ok=True)
#np.save('/Users/timothyliu/hackathon/quarter/texture_data_np_array',quarter_texture_data_list)
#np.save('/Users/timothyliu/hackathon/quarter/standard_data_np_array', quarter_standard_list)
#np.save('/Users/timothyliu/hackathon/quarter/extra_standard_data_np_array', quarter_extra_standard_list)

In [26]:
quarter_texture_data_list.shape

(688, 200, 200)

In [11]:
f_dict = file_dict(f_list)

In [ ]:
# full imgs
texture_data_list = build_full_img_list(f_list)
standard_list = build_full_img_list(std_list)
extra_standard_list = build_full_img_list(add_list)

# save data
#os.makedirs('/Users/timothyliu/hackathon/full/', exist_ok=True)
#np.save('/Users/timothyliu/hackathon/full/texture_data_np_array', texture_data_list)
#np.save('/Users/timothyliu/hackathon/full/standard_data_np_array', standard_list)
#np.save('/Users/timothyliu/hackathon/full/extra_standard_data_np_array', extra_standard_list)

In [55]:
print('=== For data set1: 172*4 textured data and 26*4 untextured data ===')
# get file label
labels = np.load('/Users/timothyliu//hackathon/labels.npy')
predicted = np.load('/Users/timothyliu//hackathon/predicted_output.npy')
#print(labels.shape,predicted.shape)

machine_guess_textured = np.mean(predicted[:299,0], axis=0)
machine_guess_untextured = np.mean(predicted[299:,1], axis=0)
print("accuracy of textured testing set = {}".format(machine_guess_textured))
print("accuracy of untextured testing set = {}".format(machine_guess_untextured))

# false positive ind: texture but classified as untextured
texture_range = 299
fp_mean_eps_1 = np.mean(predicted[:299,1], axis=0)
fn_mean_eps_1 = np.mean(predicted[299:,0], axis=0)
fp_list_1 = []
for i in range(texture_range):
    predict_label = predicted[i]
    if predict_label[1] > fp_mean_eps_1:
        fp_list_1.append(i)
        
# false negative ind: untextured but classified as textured
fn_list_1 = []
for i in range(texture_range,np.shape(predicted)[0]):
    predict_label = predicted[i]
    if predict_label[0] > fn_mean_eps_1:
        fp_list_1.append(i)
        
print('fp_list = {}'.format(fp_list_1))
print('fn_list = {}'.format(fn_list_1))
print('fp_mean_eps = {}'.format(fp_mean_eps_1))
print('fn_mean_eps = {}'.format(fn_mean_eps_1))

=== For data set1: 172*4 textured data and 26*4 untextured data ===
accuracy of textured testing set = 0.9811405745836405
accuracy of untextured testing set = 0.9731687020399783
fp_list = [8, 12, 29, 31, 36, 37, 38, 39, 43, 47, 173, 175, 177, 179, 299, 324, 399]
fn_list = []
fp_mean_eps = 0.018859425577804283
fn_mean_eps = 0.026831298181326103


In [59]:
print('wrongly labeled file name')
wrong_file = []
for el in fp_list_1:
    _ind = int(el/4)
    wrong_file.append(f_dict[str(_ind)])
w

['AuOnCu_bkg_RT-00002.tif', 'AuOnCu_bkg_RT-00003.tif', 'AuOnCu_bkg_RT-00007.tif', 'AuOnCu_bkg_RT-00007.tif', 'AuOnCu_bkg_RT-00010.tif', 'AuOnCu_bkg_RT-00010.tif', 'AuOnCu_bkg_RT-00010.tif', 'AuOnCu_bkg_RT-00010.tif', 'AuOnCu_bkg_RT-00011.tif', 'AuOnCu_bkg_RT-00012.tif', 'CL-PF-3_RT-00005.tif', 'CL-PF-3_RT-00005.tif', 'CL-PF-3_RT-00006.tif', 'CL-PF-3_RT-00006.tif', 'CL-TF-2_RT-00003.tif', 'CL-TF-2_RT-00010.tif', 'Cu_in_polyester_225um_bkg_final-00009.tif']


In [27]:
# get file label
labels = np.load('/Users/timothyliu//hackathon/labels_crpped.npy')
predicted = np.load('/Users/timothyliu//hackathon/predicted_output_cropped.npy')
print(labels.shape,predicted.shape)

machine_guess_textured = np.mean(predicted[:299,0], axis=0)
machine_guess_untextured = np.mean(predicted[299:,1], axis=0)
print("accuracy of textured testing set = {}".format(machine_guess_textured))
print("accuracy of untextured testing set = {}".format(machine_guess_untextured))

# false positive ind: texture but classified as untextured
texture_range = 999
fp_mean_eps = np.mean(predicted[:999,1], axis=0)
fn_mean_eps = np.mean(predicted[999:,0], axis=0)
fp_list = []
for i in range(texture_range):
    predict_label = predicted[i]
    if predict_label[1] > fp_mean_eps:
        fp_list.append(i)
        
# false negative ind: untextured but classified as textured
fn_list = []
for i in range(texture_range,np.shape(predicted)[0]):
    predict_label = predicted[i]
    if predict_label[0] > fp_mean_eps:
        fp_list.append(i)
        
print('len cropped fp_list = {}'.format(len(fp_list)))
print('len cropped fn_list = {}'.format(len(fn_list)))
print('cropped fp_mean_eps = {}'.format(fp_mean_eps))
print('cropped fn_mean_eps = {}'.format(fn_mean_eps))

(2000, 2) (2000, 2)
0.753455182532
0.60883660159
len cropped fp_list = 307
len cropped fn_list = 0
cropped fp_mean_eps = 0.15933270794556256
cropped fn_mean_eps = 0.05077463737171928
